## Numerical Methods
### Activity 3: Interpolation

Numpy can also fit interpolating polynomials.

In [ ]:
%matplotlib notebook

# Here is some example data.

X = [1, 3, 4, 7, 10, 12]
Y =[34, 78, 11, 12, 43, 61]



# This is 6 points in the plane, so the interpolating polynomial will have degree 5 (i.e. it will be 'quintic').

print(X)
print(Y)


We can find the coefficients of the interpolating quintic using numpy. This returns the coefficients $a_5,...,a_0$ of the interpolating quintic (note the order!).

Behind the scenes the polyfit method is just performing regression with a degree 5 polynomial here. This works because with 6 data points the best fit polynomial of degree 5 will pass through every point. We can also use the polyfit method for finding the best fit line by changing 5 to 1 in the polyfit call.

In [ ]:
import numpy as np
import numpy.polynomial as poly


coefs = np.polyfit(X, Y, 5)

print(coefs)

We can plot this interpolating polynomial.

In [ ]:
import matplotlib.pyplot as plt

x_line = np.linspace(0,13,100)

# polyval is used to define y_line as the graph of the polynomial whose coeficients are 'coefs'.
# polyval assumes 'coefs' is [a_n, a_(n-1),...,a_0] (note the order).
y_line = np.polyval(coefs, x_line)

fig, ax = plt.subplots()
plt.plot(x_line, y_line, c='cornflowerblue')
plt.scatter(X,Y, c= 'orange')
plt.show()

Let's try this method with some new data and a polynomial of higher degree.

In [ ]:
X_new = np.array(range(20))
Y_new = np.append(range(10),range(10))
print(X_new)
print(Y_new)

We have 20 data points here, so the interpolating polynomial has degree 19. We can try to find the coefficients using polyfit, like we did before.

In [ ]:
coefs_new = np.polyfit(X_new, Y_new, 19)
print(coefs_new)

We get an answer, but also a warning about being 'poorly conditioned'. This is due to the way polyfit works internally. It solves this as a regression problem using linear algebra (not using divided differences or Lagrange polynomials). For high degree polynomials the matrices involved in the polyfit solution become large, and this may create problems for the computer arithmetic system while finding the solution.

The effect that this has is that if we draw the interpolating polynomial based on the parameters we calculated in the 
previous cell, we see it does not actually go through all the data points (see below). Since, mathematically, the interpolating polynomial always goes through all the data points, the problem here is how the coefficients were calculated. I.e. with the polyfit method.

In [ ]:
x_line2 = np.linspace(0,19,100000)
y_line2 = np.polyval(coefs_new, x_line2)

fig2, ax2 = plt.subplots()
ax2.plot(x_line2, y_line2, c='cornflowerblue')
ax2.scatter(X_new,Y_new, c= 'orange')
ax2.set_ylim(-20, 30)
plt.show()


We can also find the coefficients of the interpolating polynomial by writing our own Vandermonde matrix method.

In [ ]:
def vMatrix(x_values):
    n = len(x_values)
    V = []
    for x in x_values:
        row = []
        for i in range(n):
            row.append(x**i)
        V.append(row)
    return V

van = np.array(vMatrix(X_new))
params = np.array(list(reversed(np.linalg.solve(van,Y_new))))
print(params)

# Note that linalg.solve returns parameters ordered p_0,...,p_n, but for the polyval method we need them ordered p_n,...,p_0.
# This is why we use 'reverse' here.



Now we can plot the interpolating polynomial with the coefficients we calculated using the Vandermonde matrix method.

In [ ]:
fig3, ax3 = plt.subplots()
y_line3 = np.polyval(params, x_line2)
ax3.plot(x_line2, y_line3, c='cornflowerblue')
ax3.scatter(X_new,Y_new, c= 'orange')
ax3.set_ylim(-30, 40)
plt.show()

As you can see, this is totally wrong, much worse than using polyfit, which was also bad.

We should check that the code above is working properly. To do this, we apply our Vandermonde method to the data from the very first example in the notebook. We can check that the coefficients produced are the same as those we got using the polyfit method.


In [ ]:
van2 = np.array(vMatrix(X))
params2 = np.array(list(reversed(np.linalg.solve(van2,Y))))
print(params2)

fig4, ax4 = plt.subplots()
y_line4 = np.polyval(params2, x_line)
ax4.plot(x_line, y_line, c='cornflowerblue')
ax4.scatter(X,Y, c= 'orange')
plt.show()

This looks correct. So the Vandermonde matrix method works when the degree of the interpolating polynomial is relatively small, but it has problems as the degree increases. This is what we would expect, given what we know about problems with the precision of calculated parameter values using the Vandermonde matrix method. Also, like polyfit, linalg.solve has some specific problems with large matrices due to the way it works internally. This is probably what is causing most of the problems here.

To get around these problems with polyfit and linalg.solve, we can write a method that calculates the divided differences. This method takes the $x$ values and the $y$ values and outputs the divided difference coefficients $b_0,b_1,...,b_n$.

In [ ]:
def ddiff(x_values, y_values):
    ddif_table = [[]]
    for y in y_values:
        ddif_table[0].append(y)
    for i in range(1, len(x_values)):
        column = []
        for j in range(0, len(x_values) - i):
            column.append(
                (ddif_table[i-1][j] - ddif_table[i-1][j+1]) /
                (x_values[j] - x_values[j+i]))
        ddif_table.append(column)
    return [dd[0] for dd in ddif_table]

b_values = ddiff(X_new,Y_new)
print(b_values)


Now we can use these $b$ values to find and draw the interpolating polynomial for X_new and Y_new properly. This method calculates the value at 'x_var' of a polynomial in Newton's form. Here 'x_values' are the x values from the data, and 'params' are the b values.

In [ ]:
def Npoly(x_var, x_values, params): 
    result = params[0]
    for i in range(1, len(params)):
        block = 1
        for j in range(0, i):
            block *= x_var - float(x_values[j])
        result += params[i]*block
    return result

fig5, ax5 = plt.subplots()
y_line5 = Npoly(x_line2, X_new, b_values)
plt.plot(x_line2, y_line5)
plt.plot(X_new, Y_new, marker = 'o', linestyle = 'none')
ax5.set_ylim(-20, 30)
plt.show()    

We see that this passes through all the points properly. This is the correct interpolating polynomial for this data. Comparing this to the line drawn using polyfit earlier, we see how wrong that first attempt was. Observe also that Runge's phenomenon is occurring here.

We can now calculate the interpolating polynomial for X_new and Y_new using Largange polynomials. This method finds value of the interpolating polynomial at x_var by using Lagranges form.

In [ ]:
def Lpoly(x_var, x_values, y_values): 
    result = 0
    for i in range(0, len(x_values)):
        block = 1
        for j in range(0, len(x_values)):
            if i != j:
                block *= (x_var - x_values[j]) / (x_values[i] - x_values[j])
        result += y_values[i]*block
    return result

fig6, ax6 = plt.subplots()
y_line6 = Lpoly(x_line2, X_new, Y_new)
plt.plot(x_line2, y_line6)
plt.plot(X_new, Y_new, marker = 'o', linestyle = 'none')
ax6.set_ylim(-20, 30)
plt.show()

We see that this agrees with the result of using Newton's divided difference method. This is always going to be true, because they are both ways of finding the interpolating polynomial, which is unique.

Now you should experiment with the various methods we have used here to draw the interpolating polynomial for the following data. As we only have 7 points the interpolating polynomial will have degree 6, and so every method will work. 

What you have to do is fill in the necessary details to correctly find y_line7. If you do this correctly the right line will be drawn. You should do this for every method we have used, to make sure you understand them. The result should be the same every time, as the interpolating polynomial is unique.

In [ ]:
Xt = [5,10,15,20,25,30,35]
Yt = [145,500,323,98,87,273,81]


x_line3 = np.linspace(5,35,1000)

y_line7 = #TODO

fig7, ax7 = plt.subplots()
plt.plot(x_line3, y_line7)
plt.plot(Xt, Yt, marker = 'o', linestyle = 'none')
plt.show()